## Load Data/Load wordEmbeddings/Balance Data

In [5]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
os.chdir('/home/ubuntu/Notebooks/capstone2/src')
from training_data_cleaning import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
import nltk
import warnings
import pickle

os.chdir('/home/ubuntu/Notebooks/capstone2/data')
hotels = pd.read_pickle('hotels.pickle').drop_duplicates()
os.chdir('/home/ubuntu/Notebooks/data/ta')
ta_data = pd.read_pickle('ta_data2.pickle').replace(1.0,2.0).replace(0.0,2.0).sample(frac=.5,random_state=3)
hotels = hotel_data_cleaning(hotels)
ta_data = ta_data_cleaning(ta_data)
X = pd.concat([ta_data,hotels],axis=0).reset_index().drop('index',axis=1)
X,y = balance_df_comb(X,size=18264)

In [ ]:
os.chdir('/home/ubuntu/Notebooks/data')
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',
                                                 binary=True,
                                                 limit=2000000)

### Tokenizing and NN Preprocessing 

In [7]:
x_train,x_valid,y_train,y_valid = train_test_split(X,y,test_size=.30,shuffle=True)

maxlen = 150
training_samples = x_train.shape[0]
validation_samples = x_valid.shape[0]
batch_size = 25
embedding_dims = 300
epochs = 2
embedding_dim = 300

In [8]:
token = Tokenizer(char_level=False,lower=True)
token.fit_on_texts(X)
# tokenize.texts_to_sequences(x_train.iloc[0])
x_train_seq = pad_sequences(token.texts_to_sequences(x_train),maxlen=maxlen)
x_valid_seq = pad_sequences(token.texts_to_sequences(x_valid),maxlen=maxlen)

word_index = token.word_index
max_words = len(word_index)

In [26]:
embedding_matrix = np.zeros((max_words,embedding_dims))
for word,i in word_index.items():
    if i < max_words:
        try:
            embedding_vector = word_vectors.get_vector(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        except:
            pass

from sklearn.preprocessing import OneHotEncoder 
ohe = OneHotEncoder()
y_train_new = ohe.fit_transform(y_train.values.reshape(-1,1))
y_valid_new = ohe.fit_transform(y_valid.values.reshape(-1,1))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warn

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding

model3 = Sequential()
model3.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model3.add(Dropout(rate=.2))
model3.add(LSTM(batch_size,return_sequences=False))
model3.add(Dense(4,activation='softmax'))
model3.layers[0].set_weights([embedding_matrix])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [28]:
model3.compile(optimizer='adam',
             loss='categorical_crossentropy',metrics=['acc'])
history = model3.fit(x_train_seq,y_train_new.toarray(),epochs=epochs,
                     batch_size=batch_size,
                     validation_data=(x_valid_seq,
                                      y_valid_new))

Instructions for updating:
Use tf.cast instead.
Train on 51139 samples, validate on 21917 samples
Epoch 1/2
51139/51139 [==============================] - 523s 10ms/step - loss: 0.9606 - acc: 0.5671 - val_loss: 0.9106 - val_acc: 0.5972
Epoch 2/2
51139/51139 [==============================] - 516s 10ms/step - loss: 0.7747 - acc: 0.6686 - val_loss: 0.8756 - val_acc: 0.6035


In [29]:
os.chdir('/home/ubuntu/Notebooks/capstone/models')
model3.save('first_model_combined.h5')
os.getcwd()

f = open("word_dict_combined.pickle","wb")
pickle.dump(word_index,f)
f.close()